In [6]:
import os
import glob
from PIL import Image
import imagehash
import numpy as np
import cv2
from tqdm import tqdm

In [46]:
paths = sorted(glob.glob(os.path.join('ext', '*.png')))
pil_imgs = [Image.open(p) for p in paths]

In [47]:
hashes = [imagehash.phash(img, hash_size=16) for img in pil_imgs]

In [51]:
# --- LOAD ALL FRAMES ---
folder = 'ext'
paths = sorted(glob.glob(os.path.join(folder, '*.png')))
imgs = [cv2.imread(p) for p in paths]
N = len(imgs)

In [78]:
import os
import glob
import cv2
import numpy as np
import shutil

# Configuration
SRC_FOLDER      = 'ext'        # your input folder
DST_FOLDER      = 'sorted'   # where to copy sorted frames
TOP_FRAC        = 1.0                    # fraction of height for sky crop
WHITE_THRESH    = 253                    # V-channel threshold for “white”
MIN_AREA_FRAC   = 0.003                  # min fraction of crop area to count as text

os.makedirs(DST_FOLDER, exist_ok=True)

def detect_white_text(img):
    """Return True if img’s top region has a large white overlay."""
    h, w = img.shape[:2]
    hsv  = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    v    = hsv[:,:,2]
    mask = (v >= WHITE_THRESH).astype(np.uint8)
    return (mask.sum() / mask.size) >= MIN_AREA_FRAC

BAND_FRAC       = 0.2     # top 40% of the image
MASK_MID_FRAC   = 0.6     # mask out the central 50% of width

# Percentile range to average
PCT_MIN         = 80      # e.g. 80th percentile
PCT_MAX         = 90      # e.g. 99th percentile
NUM_PCTS        = 20      # number of percentiles between min and max

# Sky‑color HSV ranges (OpenCV H:0–179, S:0–255, V:0–255)
HUE_MIN, HUE_MAX = 80, 140
SAT_MIN          = 30
V_DARK_MAX       = 50

def estimate_sky_time_filtered(img):
    """
    Estimate time by the mean of several high percentiles of V
    over sky‑colored pixels in the top band, masking out center.
    """
    h, w = img.shape[:2]
    # 1) crop top band
    top = img[0:int(h * BAND_FRAC), :, :]
    hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    h_chan, s_chan, v_chan = cv2.split(hsv)

    # 2) mask out center vertical strip
    left  = int(w * (1 - MASK_MID_FRAC) / 2)
    right = int(w - left)
    h_chan[:, left:right] = 0
    s_chan[:, left:right] = 0
    v_chan[:, left:right] = 0

    # 3) sky‑color mask
    blue_mask = ((h_chan >= HUE_MIN) & (h_chan <= HUE_MAX)
                 & (s_chan >= SAT_MIN))
    dark_mask = (v_chan <= V_DARK_MAX)
    sky_mask  = blue_mask | dark_mask

    # 4) collect V values
    sky_vals = v_chan[sky_mask]
    if sky_vals.size == 0:
        # fallback to unmasked band
        sky_vals = v_chan[v_chan > 0]

    # 5) compute a range of percentiles and return their mean
    pcts = np.linspace(PCT_MIN, PCT_MAX, NUM_PCTS)
    vals = np.percentile(sky_vals, pcts)
    return vals.mean()

# 2) Filter to those with white text and compute times
text_frames = []
for p, img in zip(paths, imgs):
    if img is None:
        continue
    if detect_white_text(img):
        t = estimate_sky_time_filtered(img)
        text_frames.append((p, t))

# 3) Sort by descending brightness (brightest = earliest)
text_frames.sort(key=lambda x: -x[1])

# 4) Copy into DST_FOLDER with zero‑padded sequence names
for idx, (src_path, _) in enumerate(text_frames, start=1):
    dst_name = f"{idx:04d}.png"
    dst_path = os.path.join(DST_FOLDER, dst_name)
    shutil.copy2(src_path, dst_path)

print(f"Copied {len(text_frames)} sorted text‑frames into '{DST_FOLDER}'.")


Copied 123 sorted text‑frames into 'sorted'.


In [1]:
import cv2
import numpy as np
import glob
import os

from sklearn.manifold import Isomap

# --------------------------------------------------
# 1) Load frames from the folder "ext/" in random order
# --------------------------------------------------

# Grab all image paths (change the extension if needed)
frame_paths = glob.glob("ext/*.png")  # or *.jpg, *.bmp, etc.

# Optional: Shuffle them to simulate the "randomly shuffled" scenario
# If your folder is already random, you can skip this step
np.random.shuffle(frame_paths)

print("Number of frames found:", len(frame_paths))

# Load images in the given order
images = [cv2.imread(p) for p in frame_paths]
h, w, c = images[0].shape  # height, width, channels (for writing video later)

# --------------------------------------------------
# 2) Feature extraction
# --------------------------------------------------
# A simple approach: resize and flatten each frame into a vector.
# You may want to use more sophisticated descriptors or a deeper 
# feature extraction (e.g. from a CNN) for better results.

resized_dim = (320, 320)  # Small dimension to reduce computational cost
feature_vectors = []

for img in images:
    resized = cv2.resize(img, resized_dim)
    # Flatten into a single row (64*64*3)
    vec = resized.flatten()
    feature_vectors.append(vec)

feature_vectors = np.array(feature_vectors, dtype=np.float32)
print("Feature vector shape:", feature_vectors.shape)

# --------------------------------------------------
# 3) Apply Isomap for dimensionality reduction
# --------------------------------------------------
# We'll embed to 1 dimension (n_components=1).
# This typically forces Isomap to arrange the frames along a single 'time-like' axis.
# n_neighbors should be chosen based on how smooth the transitions are between frames. 
# You may need to experiment with these parameters:

n_neighbors = 5
isomap = Isomap(n_neighbors=n_neighbors, n_components=1)
embedding_1d = isomap.fit_transform(feature_vectors)

# embedding_1d is shape (num_frames, 1)
print("Isomap embedding shape:", embedding_1d.shape)

# --------------------------------------------------
# 4) Sort frames according to their Isomap coordinate
# --------------------------------------------------
frame_order = np.argsort(embedding_1d[:, 0])
print("Proposed new order of frames:", frame_order)

# Reorder the images
images_reordered = [images[i] for i in frame_order]

# --------------------------------------------------
# 5) Save the reordered frames as a video
# --------------------------------------------------

output_video_path = "reconstructed.avi"
fps = 30.0  # or match your original framerate

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

for frame in images_reordered:
    out.write(frame)
out.release()

print("Reconstructed video written to:", output_video_path)


Number of frames found: 3940
Feature vector shape: (3940, 307200)


C:\Users\shuce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
C:\Users\shuce\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


Isomap embedding shape: (3940, 1)
Proposed new order of frames: [ 818  362 2115 ... 2670 1241 1607]
Reconstructed video written to: reconstructed.avi


In [2]:
output_video_path = "reconstructed.avi"
fps = 30.0  # or match your original framerate

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

for frame in images_reordered:
    out.write(frame)
out.release()

print("Reconstructed video written to:", output_video_path)

Reconstructed video written to: reconstructed.avi
